# Installs & imports

In [1]:
!pip install xgboost
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00


In [145]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import ndcg_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.xgboost import WandbCallback
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [5]:
%cd gdrive/MyDrive/work/test

/content/gdrive/MyDrive/work/test


# Dataset

In [160]:
data = pd.read_csv('intern_task.csv', header=0)
data.shape

(235258, 146)

In [161]:
data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000


In [162]:
data['rank'].value_counts()

rank
0    121521
1     75815
2     31910
3      4209
4      1803
Name: count, dtype: int64

In [163]:
data.describe()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
count,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,...,2.352580e+05,2.352580e+05,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000
mean,0.677869,14828.413401,1.911960,0.206233,1.189847,0.550272,1.960082,0.803964,0.097557,0.541174,...,4.763432e+02,1.046698e+04,0.070299,0.740710,11.067947,0.281747,0.027033,0.000269,22.457910,9.560379
std,0.830033,8193.945170,1.237374,0.579089,1.037233,0.790947,1.203534,0.339955,0.266035,0.419973,...,1.954840e+04,2.649584e+06,0.100721,0.352800,15.336861,0.392089,0.033351,0.002088,63.708018,14.347378
min,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,8215.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.666667,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.620522,0.000000,0.000000,0.006703,0.000000,4.000000,2.000000
50%,0.000000,14935.000000,2.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.500000,...,0.000000e+00,0.000000e+00,0.000000,0.926779,9.448900,0.000000,0.017761,0.000008,12.000000,5.500000
75%,1.000000,21580.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1.000000,...,1.000000e+00,4.000000e-01,0.142857,0.999613,16.146733,0.500000,0.034954,0.000109,28.000000,12.000000
max,4.000000,29995.000000,31.000000,18.000000,27.000000,9.000000,31.000000,1.000000,1.000000,1.000000,...,1.731553e+06,8.000000e+08,0.818182,1.000000,2506.799764,1.000000,0.711261,0.250000,15074.000000,1552.000000


У данных очень большой разброс, поэтому надо будет их нормализовать после разбиения на трейн и тест

In [164]:
data.rename(columns={'query_id': 'qid'}, inplace=True)  # for XGBRanker

## Tran-val-test split

In [165]:
gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
train_ids, valtest_ids = next(gss.split(data, groups=data.qid))
train_data, valtest_data = data.loc[data.index[train_ids]], data.loc[data.index[valtest_ids]]

In [166]:
feature_columns = train_data.columns[2:]
scaler = MinMaxScaler()

In [167]:
train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])
valtest_data[feature_columns] = scaler.transform(valtest_data[feature_columns])

In [168]:
train_data.describe()

,rank,qid,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
count,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,...,1.889890e+05,1.889890e+05,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000,188989.000000
mean,0.679585,14717.613300,0.061601,0.011544,0.043787,0.061254,0.063130,0.804522,0.098809,0.540538,...,2.904077e-04,1.628371e-05,0.086166,0.743419,0.004388,0.284149,0.037986,0.001060,0.001493,0.006162
std,0.832516,8125.953562,0.040194,0.032317,0.038375,0.087792,0.039117,0.340545,0.268179,0.421195,...,1.151542e-02,3.695227e-03,0.123108,0.351603,0.006189,0.394309,0.047097,0.008538,0.004588,0.009404
min,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,8275.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.666667,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.629084,0.000000,0.000000,0.009373,0.000000,0.000265,0.001289
50%,0.000000,14845.000000,0.064516,0.000000,0.037037,0.000000,0.064516,1.000000,0.000000,0.500000,...,0.000000e+00,0.000000e+00,0.000000,0.929462,0.003726,0.000000,0.024909,0.000028,0.000796,0.003544
75%,1.000000,21250.000000,0.096774,0.000000,0.074074,0.111111,0.096774,1.000000,0.000000,1.000000,...,5.775163e-07,6.250000e-10,0.174603,0.999899,0.006418,0.500000,0.049045,0.000420,0.001858,0.007732
max,4.000000,29995.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [169]:
gss_val = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=42)
val_ids, test_ids = next(gss_val.split(valtest_data, groups=valtest_data.qid))
val_data, test_data = valtest_data.loc[valtest_data.index[val_ids]], valtest_data.loc[valtest_data.index[test_ids]]

In [170]:
X_train, X_val, X_test = train_data.drop(['rank', 'qid'], axis=1), val_data.drop(['rank', 'qid'], axis=1), test_data.drop(['rank'], axis=1)
y_train, y_val, y_test = train_data['rank'], val_data['rank'], test_data['rank']
qid_train, qid_val = train_data['qid'], val_data['qid']

X_train.shape, X_val.shape, X_test.shape

((188989, 144), (22602, 144), (23667, 145))

# Training

In [171]:
wandb.login()

True

In [176]:
def wandb_train_model():
    wandb.init(project='vk_test')

    model = xgb.XGBRanker(
        tree_method="hist",
        objective="rank:ndcg",
        lambdarank_pair_method="topk",
        eval_metric='ndcg@5',
        device=device,

        max_depth=wandb.config.max_depth,
        learning_rate=wandb.config.learning_rate,
        n_estimators=wandb.config.n_estimators,
    )

    model.fit(X_train,
                y_train,
                qid=qid_train,
                eval_set=[(X_val, y_val)],
                eval_qid=[qid_val],
                callbacks=[WandbCallback(log_model=True)]
              )

    ndcg_val = model.score(X_test, y_test)
    wandb.log({"ndcg@5": ndcg_val})

In [177]:
sweep_config = {
    'method': 'bayes',
    "metric": {"name": "ndcg@5", "goal": "maximize"},
    'parameters': {
        'learning_rate': {'min': 0.01, 'max': 0.3},
        'max_depth': {'min': 3, 'max': 10},
        'n_estimators': {'min': 50, 'max': 300}
    }
}

In [178]:
sweep_id = wandb.sweep(sweep_config, project="vk_test_ranking")
agent = wandb.agent(sweep_id=sweep_id, function=wandb_train_model, count=20)

Create sweep with ID: 0fggvwl7
Sweep URL: https://wandb.ai/vknyazkova/vk_test_ranking/sweeps/0fggvwl7


wandb: Agent Starting Run: m5tdcnj1 with config:
wandb: 	learning_rate: 0.1604390643734062
wandb: 	max_depth: 7
wandb: 	n_estimators: 229
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ndcg@5,▁
validation_0-ndcg@5,▁▂▄▄▅▆▆▆▆▇▇▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████████████▇▇
epoch,228
ndcg@5,0.45047
validation_0-ndcg@5,0.50418


wandb: Agent Starting Run: dkq7cl78 with config:
wandb: 	learning_rate: 0.031021577230791353
wandb: 	max_depth: 10
wandb: 	n_estimators: 152
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ndcg@5,▁
validation_0-ndcg@5,▁▃▅▆▇▇▇▇▇▇▇▇▇▇▇███████████▇██▇█▇▇▇▇▇██▇▇
epoch,232
ndcg@5,0.45286
validation_0-ndcg@5,0.47879


wandb: Agent Starting Run: krtzg71k with config:
wandb: 	learning_rate: 0.16182322407970548
wandb: 	max_depth: 3
wandb: 	n_estimators: 227
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ndcg@5,▁
validation_0-ndcg@5,▁▂▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇███████████████████████
epoch,226
ndcg@5,0.46003
validation_0-ndcg@5,0.48934


Отчет по подбору гиперпараметров [тут](https://wandb.ai/vknyazkova/vk_test_ranking/reports/XGBRanker-hp-tuning--Vmlldzo3NzQ5MDE5?accessToken=15b6je4xaftatwujbqyqy4heio2j84ygy1wxdxpe9lkqeibiocf4mf9iftn7cz7x)

# Evaluation

Выбрала лучшие параметры по результатам экспериментов выше:

In [179]:
best_model = xgb.XGBRanker(
        tree_method="hist",
        objective="rank:ndcg",
        lambdarank_pair_method="topk",
        lambdarank_num_pair_per_sample=8,
        eval_metric='ndcg@5',
        device=device,

        max_depth=4,
        learning_rate=0.1982,
        n_estimators=232,
    )

best_model.fit(X_train,
               y_train,
               qid=qid_train)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None,
          device=device(type='cuda'), early_stopping_rounds=None,
          enable_categorical=False, eval_metric='ndcg@5', feature_types=None,
          gamma=None, grow_policy=None, importance_type=None,
          interaction_constraints=None, lambdarank_num_pair_per_sample=8,
          lambdarank_pair_method='topk', learning_rate=0.1982, max_bin=None,
          max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None,
          max_depth=4, max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=232,
          n_jobs=None, ...)

In [181]:
score = best_model.score(valtest_data.drop(['rank'], axis=1), valtest_data['rank'])
print(f'{best_model.eval_metric}: {score}')

ndcg@5: 0.48200503775589404
